# Scripts Python - Project : Mapping Cerebral Network Alterations in CKD

**Léa Payre**

M2 DigiPharm - *Aix Marseille Université*

# Statistiques descriptives 

## Caractérisation des données de captation

In [ ]:
import pandas as pd
import numpy as np

fichier = "CTRL_DTPA.xlsx"  # à modifier par groupe x traceur

df = pd.read_excel(fichier) 

regions = df.columns[1:] # première colonne = Sujet_ID et les suivantes = régions

# fonction statistiques
def stats_intra_sujet(valeurs):
    return pd.Series({
        "Moyenne": np.mean(valeurs),
        "Médiane": np.median(valeurs),
        "Ecart_type": np.std(valeurs, ddof=1),
        "Min": np.min(valeurs),
        "Max": np.max(valeurs),
        "IQR": np.percentile(valeurs, 75) - np.percentile(valeurs, 25)
    })

# appliquer la fonction sur chaque individu
stats_individus = df.apply(
    lambda row: stats_intra_sujet(row[regions].values.astype(float)),
    axis=1
)

# ajouter ID sujet
stats_individus.insert(0, "Sujet_ID", df["Sujet_ID"])

# exportation 
nom_sortie = fichier.replace(".xlsx", "_stats_individus.xlsx")
stats_individus.to_excel(nom_sortie, index=False)

print(f" Statistiques descriptives intra-sujet sauvegardées dans {nom_sortie}")

## Analyses inter-sujets par région cérébrale

In [ ]:
import pandas as pd
import numpy as np

fichier = "CTRL_DTPA.xlsx" 

df = pd.read_excel(fichier)

regions = df.columns[1:]

df[regions] = df[regions].apply(pd.to_numeric, errors="coerce")
 
def iqr(s):  # IQR = Q3 - Q1
    return np.percentile(s.dropna(), 75) - np.percentile(s.dropna(), 25)

def cv(s):   # coefficient de variation
    m = s.mean()
    return np.nan if m == 0 else (s.std(ddof=1) / m) * 100

#  statistiques par groupe/region 
stats_grp = pd.DataFrame({
    "N":        df[regions].count(),               # nb valeurs non-NaN
    "Nb_Na":    df[regions].isna().sum(),          # nb NaN
    "Moyenne":  df[regions].mean(),
    "Médiane":  df[regions].median(),
    "Ecart_type": df[regions].std(ddof=1),
    "Min":      df[regions].min(),
    "Max":      df[regions].max(),
    "IQR":      df[regions].apply(iqr),
    "Skewness": df[regions].skew(),                # asymétrie
    "CV_%":     df[regions].apply(cv)              # coefficient de variation en %
})

# régions en colonne 
stats_grp = stats_grp.reset_index().rename(columns={"index": "Region"})

nom_sortie = fichier.replace(".xlsx", "_stats_groupes.xlsx")
stats_grp.to_excel(nom_sortie, index=False)

print(f" Statistiques descriptives par groupe sauvegardées dans {nom_sortie}")

# Évaluation de la normalité des distribution (Shapiro-Wilk)

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

fichier_excel = "CTRL_DTPA_Mbq.xlsx" 

noms_roi = [
    "Olfactory_Bulb", "WM_Cerebellum", "Ventricle_III_IV",
    "Ventricle_Lateral", "Brain_Stem", "WM_Ctx",
    "Ctx_Parietal", "Ctx_Occipital", 
    "tx_Temporal_L", "Ctx_Frontal", 
    "Ctx_Cingulate", "Cerebellum", 
    "Amygdala", "Hippocampus",
    "Thalamus_Hypothalamus", "Caudate_Putamen",
]

# si ligne 1 = en-tête avec noms des sujets
df = pd.read_excel(fichier_excel, header=None) 

# vérification du fichier
nb_lignes_data = df.shape[0]
print(f"  Nombre de lignes dans le fichier : {nb_lignes_data}")
print(f"  Nombre de ROI attendues : {len(noms_roi)}")

# shapiro-wilk pour chaque ROI
print(f"TEST DE SHAPIRO-WILK - {fichier_excel}")
resultats = []

for idx in range(min(len(noms_roi), nb_lignes_data)):
    nom_roi = noms_roi[idx]
    
    # extraction des valeurs de la ligne idx : toutes les colonnes = tous les sujets
    valeurs = df.iloc[idx, :].values
    
    # convertion en numerique et suppression des valeurs manquantes
    valeurs_numeriques = pd.to_numeric(valeurs, errors='coerce')
    valeurs_propres = valeurs_numeriques[~np.isnan(valeurs_numeriques)]
    
    n = len(valeurs_propres)
    
    # test de Shapiro-Wilk
    if n >= 3:
        statistic, p_value = stats.shapiro(valeurs_propres)
        
        # statistiques descriptives
        moyenne = np.mean(valeurs_propres)
        mediane = np.median(valeurs_propres)
        ecart_type = np.std(valeurs_propres, ddof=1)
        q1 = np.percentile(valeurs_propres, 25)
        q3 = np.percentile(valeurs_propres, 75)
        skewness = stats.skew(valeurs_propres)
        kurtosis = stats.kurtosis(valeurs_propres, fisher=True)
        
        # decision normalité
        normale = " Normale" if p_value > 0.05 else " Non-normale"
        
        # stockage
        resultats.append({
            'ROI': nom_roi,
            'n': n,
            'Moyenne': moyenne,
            'Mediane': mediane,
            'Ecart-type': ecart_type,
            'Q1': q1,
            'Q3': q3,
            'Skewness': skewness,
            'Kurtosis': kurtosis,
            'W_statistic': statistic,
            'p_value': p_value,
            'Normalite': normale
        })
        
        # affichage
        print(f"{idx+1:2d}. {nom_roi:<30s} | n={n:2d} | W={statistic:.4f} | p={p_value:.5f} | {normale}")
    
    else:
        print(f"{idx+1:2d}. {nom_roi:<30s} | n={n:2d} |   Effectif insuffisant")
        resultats.append({
            'ROI': nom_roi,
            'n': n,
            'Moyenne': np.nan,
            'Mediane': np.nan,
            'Ecart-type': np.nan,
            'Q1': np.nan,
            'Q3': np.nan,
            'Skewness': np.nan,
            'Kurtosis': np.nan,
            'W_statistic': np.nan,
            'p_value': np.nan,
            'Normalite': 'N/A'
        })

# résumé
print("\n" + "="*110)
print(" RÉSUMÉ")
print("="*110)

df_resultats = pd.DataFrame(resultats)

# comptage des ROI normales
nb_normales = df_resultats[df_resultats['p_value'] > 0.05].shape[0]
nb_totales = len(df_resultats)
pct_normales = (nb_normales / nb_totales * 100) if nb_totales > 0 else 0

print(f" ROI normales (p > 0.05)     : {nb_normales}/{nb_totales} ({pct_normales:.1f}%)")
print(f" ROI non-normales (p ≤ 0.05) : {nb_totales - nb_normales}/{nb_totales}")

# statistiques sur Skewness et Kurtosis
skew_anormal = df_resultats[(df_resultats['Skewness'].abs() > 1)].shape[0]
kurt_anormal = df_resultats[(df_resultats['Kurtosis'].abs() > 2)].shape[0]

print(f"\n  ROI avec |Skewness| > 1    : {skew_anormal}/{nb_totales}")
print(f"  ROI avec |Kurtosis| > 2    : {kurt_anormal}/{nb_totales}")

# export en excel
nom_sortie = fichier_excel.replace('.xlsx', '_resultats_shapiro.xlsx')
df_resultats.to_excel(nom_sortie, index=False)
print(f"\n Résultats exportés dans : {nom_sortie}")

print("\n" + "="*110)
print(" TABLEAU DÉTAILLÉ DES RÉSULTATS DU GROUPE CONTRÔLE, MARQUEUR 99mTc-DTPA")
print("="*110)

# affichage formaté
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

print(df_resultats[['ROI', 'n', 'W_statistic', 'p_value', 'Skewness', 'Kurtosis', 'Normalite']].to_string(index=False))

# Visualisations exploratoires

## Boxplots

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

fichiers_dtpa = [
    "CTRL_DTPA_Mbq.xlsx",
    "ARD_DTPA_Mbq.xlsx",
]
# importation et fusion
df_all = []
for f in fichiers_dtpa:
    df = pd.read_excel(f)
    df = df.melt(id_vars=["Sujet_ID"], var_name="Region", value_name="Valeur")
    
    # choix du groupe
    if "CTRL" in f:
        df["Groups"] = "CTRL"
    else:
        df["Groups"] = "ARD"
    
    df_all.append(df)

df_all = pd.concat(df_all, ignore_index=True)

# sanitation
df_all["Valeur"] = (
    df_all["Valeur"]
      .astype(str)
      .str.replace(",", ".", regex=False)
      .str.strip()
)
df_all["Valeur"] = pd.to_numeric(df_all["Valeur"], errors="coerce")
print("NaN restants :", df_all["Valeur"].isna().sum())

# données (par traceur) 
data_dtpa = df_all 

# palette de couleur
palette = {
    "CTRL": "#2E86AB",
    "ARD": "#E63946"
}

# boxplot
plt.figure(figsize=(24, 10))
sns.boxplot(data=data_dtpa, x="Region", y="Valeur", hue="Groups", 
            showfliers=False, palette=palette, linewidth=2.5, width=0.65)
for spine in ['bottom', 'left']:
    ax.spines[spine].set_color('#555555')  
    ax.spines[spine].set_linewidth(1.2)

for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

sns.stripplot(data=data_dtpa, x="Region", y="Valeur", hue="Groups",
              dodge=True, alpha=0.5, color="black", size=4)

# limite de l'axe Y 
plt.ylim(0, 4.5e-6)  # De 0 à 3×10⁻⁵

# labels 
plt.xticks(rotation=28, ha='right', fontsize=24, fontweight='bold')
plt.yticks(ha='right', fontsize=20, fontweight='bold')

# coloration des labels par groupe anatomique
ax = plt.gca()

# couleurs par groupe anatomique
color_map = {
    # ventricules / substance blanche
    'Ventricle_III_LV': '#555F64',
    'Ventricle_Lateral': '#555F64',
    'WM_Ctx': '#555F64',
    # tronc cerebrale
    'Brain_Stem': '#68633B',
    # structures cérébelleuses
    'Cerebellum': '#4A3C6E',
    'WM_Cerebellum': '#4A3C6E',
    # corticale
    'Ctx_Parietal': '#253D57',
    'Ctx_Occipital': '#24425C',
    'Ctx_Frontal': '#24425C',
    'Ctx_Cingulate': '#24425C',
    'tx_Temporal': '#24425C',
    # sous corticales 
    'Amygdala': '#1F4035',
    'Hippocampus': '#204035',
    'Caudate_Putamen': '#204035',
    'Thalamus_Hyppothalamus': "#204035",
    'Olfactory_Bulb': '#204035'
}

# appliquer les couleurs aux labels
for i, ticklabel in enumerate(ax.get_xticklabels()):
    roi_name = ticklabel.get_text()
    if roi_name in color_map:
        ticklabel.set_color(color_map[roi_name])
        ticklabel.set_fontweight('bold')
        

plt.xlabel("Brain Regions\n", fontsize=26, fontweight='bold')
plt.ylabel("Uptake (ID/g ×10⁻⁶)\n", fontsize=22, fontweight='bold')
plt.title("\nBlood–Brain Barrier Permeability (⁹⁹ᵐTc-DTPA) : Control vs ARD Groups\n",
          fontsize=29, fontweight='bold', pad=6)

# légende
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles[:2], labels[:2], 
           title='Groups', title_fontsize=18, fontsize=15,
           loc='upper right', frameon=True, shadow=True)
for i in range(len(ax.get_xticks()) - 1):
    ax.axvline(x=i + 0.5, color="#BBBBBB", linestyle='-', linewidth=0.6, alpha=0.4)
    
# dictionnaire de noms affichés en anglais
custom_labels = {
    "Ctx_Frontal": "Frontal Cortex",
    "Ctx_Parietal": "Parietal Cortex",
    "tx_Temporal": "Temporal Cortex",
    "Ctx_Occipital": "Occipital Cortex",
    "Ctx_Cingulate": "Cingulate Cortex",
    "Olfactory_Bulb": "Olfactory Bulb",
    "Amygdala": "Amygdala",
    "Hippocampus": "Hippocampus",
    "Thalamus_Hyppothalamus": "Thalamus – Hypothalamus",
    "Caudate_Putamen": "Caudate – Putamen",
    "Cerebellum": "Cerebellum",
    "WM_Cerebellum": "Cerebellar White Matter",
    "Brain_Stem": "Brainstem",
    "WM_Ctx": "Subcortical White Matter",
    "Ventricle_Lateral": "Lateral Ventricles",
    "Ventricle_III_IV": "III–IV Ventricles"
}

# remplacer les labels affichés sur l'axe X
ax.set_xticklabels([
    custom_labels.get(label.get_text(), label.get_text())
    for label in ax.get_xticklabels()
])
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles[:2], labels[:2],  # Garder seulement CTRL et ARD (les 2 premiers)
           title='Groups', loc='lower right', bbox_to_anchor=(1.0, 1.02), 
           frameon=True, ncol=2, fontsize=14)
plt.tight_layout()

# export en png et pdf
plt.savefig('DTPA_boxplot_CTRL_vs_ARD.png', 
            dpi=600, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.savefig('DTPA_boxplot_CTRL_vs_ARD.pdf', 
            format='pdf', bbox_inches='tight',
            facecolor='white')

plt.show()

## Heatmap

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np                         
from matplotlib import patheffects as pe   

plt.rcParams.update({
    'font.family': 'Helvetica',
    'pdf.fonttype': 42,  
})

fichiers = ["CTRL_DTPA_Mbq.xlsx", "ARD_DTPA_Mbq.xlsx"]
infos = [(f, "CTRL" if "CTRL" in f else "ARD") for f in fichiers]

df_all = []
for f, grp in infos:
    df = pd.read_excel(f)
    df = df.melt(id_vars=["Sujet_ID"], var_name="Region", value_name="Valeur")
    df["Groups"] = grp
    df_all.append(df)

df_all = pd.concat(df_all, ignore_index=True)
df_all["Valeur"] = (
    df_all["Valeur"].astype(str).str.replace(",", ".", regex=False).str.strip()
)
df_all["Valeur"] = pd.to_numeric(df_all["Valeur"], errors="coerce")

moyennes = (
    df_all.groupby(["Groups", "Region"])["Valeur"].mean().reset_index()
)

pivot = moyennes.pivot(index="Groups", columns="Region", values="Valeur").reindex(["CTRL", "ARD"])
vals = pivot.values.flatten()
vmin = pd.Series(vals).quantile(0.02)
vmax = pd.Series(vals).quantile(0.98)

ROI_ORDER = [
    "Olfactory_Bulb", "WM_Cerebellum", "Ventricle_III_IV", "Ventricle_Lateral",
    "Brain_Stem", "WM_Ctx", "Ctx_Parietal", "Ctx_Occipital", "tx_Temporal",
    "Ctx_Frontal", "Ctx_Cingulate", "Cerebellum", "Amygdala", "Hippocampus",
    "Thalamus_Hyppothalamus", "Caudate_Putamen",
]
cols = [c for c in ROI_ORDER if c in pivot.columns] + [c for c in pivot.columns if c not in ROI_ORDER]
pivot = pivot.loc[:, cols]

LABEL_MAP = {
    "Olfactory_Bulb": "Olfactory Bulb",
    "WM_Cerebellum": "Cerebellar White Matter",
    "Ventricle_III_IV": "III–IV Ventricles",
    "Ventricle_Lateral": "Lateral Ventricles",
    "Brain_Stem": "Brainstem",
    "WM_Ctx": "Subcortical White Matter",
    "Ctx_Parietal": "Parietal Cortex",
    "Ctx_Occipital": "Occipital Cortex",
    "tx_Temporal": "Temporal Cortex",
    "Ctx_Frontal": "Frontal Cortex",
    "Ctx_Cingulate": "Cingulate Cortex",
    "Cerebellum": "Cerebellum",
    "Amygdala": "Amygdala",
    "Hippocampus": "Hippocampus",
    "Thalamus_Hyppothalamus": "Thalamus – Hypothalamus",
    "Caudate_Putamen": "Caudate – Putamen",
}
pivot_disp = pivot.copy()
pivot_disp.columns = [LABEL_MAP.get(c, c) for c in pivot.columns]

#  heatmap 
plt.figure(figsize=(20, 5))
ax = sns.heatmap(
    pivot_disp, cmap="plasma", vmin=vmin, vmax=vmax,
    annot=False, cbar_kws={'label': 'Mean tracer uptake (ID/g)'}
)

# colorbar 
cbar = ax.collections[0].colorbar
cbar.ax.set_ylabel("\nMean tracer uptake (ID/g)", fontsize=14, fontweight='semibold')
cbar.outline.set_linewidth(1)
cbar.ax.tick_params(labelsize=10)

color_map = {
    # ventricules / substance blanche
    'III–IV Ventricles': '#555F64',
    'Lateral Ventricles': '#555F64',
    'Subcortical White Matter': '#555F64',
    # tronc cérébral
    'Brainstem': '#68633B',
    # structures cerebelleuses
    'Cerebellum': '#4A3C6E',
    'Cerebellar White Matter': '#4A3C6E',
    # corticales
    'Parietal Cortex': '#253D57',
    'Occipital Cortex': '#24425C',
    'Frontal Cortex': '#24425C',
    'Cingulate Cortex': '#24425C',
    'Temporal Cortex': '#24425C',
    # sous corticales
    'Amygdala': '#204035',
    'Hippocampus': '#204035',
    'Caudate – Putamen': '#204035',
    'Thalamus – Hypothalamus': "#204035",
    'Olfactory Bulb': '#204035'
}
BASE_LABEL_COLOR = "#2B3A42"  

# centrer les ticks au milieu des cellules
ax.set_xticks(np.arange(pivot_disp.shape[1]) + 0.5)

# labels avec ancrage correct
ax.set_xticklabels(
    list(pivot_disp.columns),
    rotation=28, ha="right", rotation_mode="anchor",
    fontsize=18
)

#  gras + halo blanc + couleur par catégorie
for lbl in ax.get_xticklabels():
    txt = lbl.get_text()
    lbl.set_fontweight('bold')
    lbl.set_color(color_map.get(txt, BASE_LABEL_COLOR))
    lbl.set_path_effects([pe.withStroke(linewidth=2.0, foreground="white")])

for lbl in ax.get_yticklabels():
    lbl.set_fontweight('bold')
    lbl.set_color(BASE_LABEL_COLOR)
    lbl.set_path_effects([pe.withStroke(linewidth=2.0, foreground="white")])

ax.set_yticklabels(ax.get_yticklabels(), rotation=0, ha="right", fontsize=20, fontweight="bold")

plt.title("Heatmap of Blood–Brain Barrier Permeability (⁹⁹ᵐTc-DTPA) : Control vs ARD Groups\n",
          fontsize=22, pad=6, weight='bold')
plt.xlabel("Brain Regions", fontsize=20, weight='bold')
plt.ylabel(" Groups\n", fontsize=18, weight='bold')

ax.tick_params(axis='y', labelsize=15)

plt.tight_layout()

plt.savefig('DTPA_heatmap_CTRL_vs_ARD.png', dpi=600, bbox_inches='tight', facecolor='white')
plt.savefig('DTPA_heatmap_CTRL_vs_ARD.pdf', format='pdf', bbox_inches='tight', facecolor='white')

plt.show()

# Analyse statistique intergroupe (Mann-Whitney)

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import mannwhitneyu, shapiro
from statsmodels.stats.multitest import multipletests
import warnings
warnings.filterwarnings('ignore')


def load_data(ctrl_file, ard_file, sheet_name=0):
    """
    charge les données CTRL et ARD depuis des fichiers Excel
    
    parameters:
    ctrl_file : str
        chemin vers le fichier Excel des contrôles
    ard_file : str
        chemin vers le fichier Excel des ARD
    sheet_name : str or int
        nom ou index de la feuille à charger
    
    Returns:

    df_ctrl, df_ard : pandas.DataFrame
        DataFrames contenant les données
    """
    df_ctrl = pd.read_excel(ctrl_file, sheet_name=sheet_name)
    df_ard = pd.read_excel(ard_file, sheet_name=sheet_name)
    
    print(f" Données CTRL chargées: {len(df_ctrl)} observations")
    print(f"Données ARD chargées: {len(df_ard)} observations")
    
    return df_ctrl, df_ard


def get_roi_columns(df):
    """
    extrait les colonnes correspondant aux ROIs (exclut Sujet_ID)
    
    parameters:
    df : pandas.DataFrame
        DataFrame contenant les données
    
    returns:
    list : Liste des noms de colonnes des ROIs
    """
    # on exclu la colonne Sujet_ID
    roi_cols = [col for col in df.columns if col not in ['Sujet_ID', 'Subject_ID', 'ID']]
    return roi_cols

def test_normality(data, roi_name):
    """
    Teste la normalité des données avec le test de Shapiro-Wilk
    
    parameters:
    data : array-like
        Données à tester
    roi_name : str
        Nom de la ROI (pour affichage)
    
    Returns:
    statistic, p_value : float
        statistique et p-value du test de Shapiro
    """
    if len(data) < 3:
        return None, None
    
    stat, p = shapiro(data)
    return stat, p


def mann_whitney_comparison(df_ctrl, df_ard, roi_cols, alpha=0.05, apply_fdr=True):
    """
    Effectue les tests de Mann-Whitney pour chaque ROI avec correction FDR
    
    parameters:
    df_ctrl : pandas.DataFrame
        Données du groupe contrôle
    df_ard : pandas.DataFrame
        Données du groupe ARD
    roi_cols : list
        Liste des noms de colonnes des ROIs
    alpha : float
        Seuil de significativité (défaut: 0.05)
    apply_fdr : bool
        Appliquer la correction FDR de Benjamini-Hochberg (défaut: True)
    
    Returns:
    results_df : pandas.DataFrame
        DataFrame avec les résultats statistiques pour chaque ROI
    """
    
    results = []
    
    print("\n" + "="*80)
    print("ANALYSE STATISTIQUE - TEST DE MANN-WHITNEY")
    print("="*80)
    
    for roi in roi_cols:
        # extraction des données pour cette ROI
        ctrl_data = df_ctrl[roi].dropna().values
        ard_data = df_ard[roi].dropna().values
        
        # statistiques descriptives
        ctrl_median = np.median(ctrl_data)
        ard_median = np.median(ard_data)
        ctrl_mean = np.mean(ctrl_data)
        ard_mean = np.mean(ard_data)
        
        # tests de normalité
        ctrl_shapiro_stat, ctrl_shapiro_p = test_normality(ctrl_data, f"{roi}_CTRL")
        ard_shapiro_stat, ard_shapiro_p = test_normality(ard_data, f"{roi}_ARD")
        
        # test de Mann-Whitney
        statistic, p_value = mannwhitneyu(ctrl_data, ard_data, alternative='two-sided')
        
        # calcul de l'effect size (r = Z / sqrt(N))
        n1, n2 = len(ctrl_data), len(ard_data)
        N = n1 + n2
        # approximation de Z à partir de U
        U = statistic
        mu_U = n1 * n2 / 2
        sigma_U = np.sqrt(n1 * n2 * (N + 1) / 12)
        Z = (U - mu_U) / sigma_U
        effect_size = abs(Z) / np.sqrt(N)
        
        results.append({
            'ROI': roi,
            'n_CTRL': n1,
            'n_ARD': n2,
            'Median_CTRL': ctrl_median,
            'Median_ARD': ard_median,
            'Mean_CTRL': ctrl_mean,
            'Mean_ARD': ard_mean,
            'Shapiro_p_CTRL': ctrl_shapiro_p,
            'Shapiro_p_ARD': ard_shapiro_p,
            'U_statistic': statistic,
            'p_value': p_value,
            'effect_size_r': effect_size
        })
    
    # creation du DataFrame des résultats
    results_df = pd.DataFrame(results)
    
    # correction FDR 
    if apply_fdr:

        reject, pvals_corrected, alphacSidak, alphacBonf = multipletests(
            results_df['p_value'], 
            alpha=alpha, 
            method='fdr_bh'
        )
        results_df['p_value_FDR'] = pvals_corrected
        results_df['Significant_FDR'] = reject
    else:
        results_df['Significant'] = results_df['p_value'] < alpha
    
    # symboles de significativité
    def get_significance_symbol(p, fdr=apply_fdr):
        p_val = p if not fdr else p
        if p_val < 0.001:
            return '***'
        elif p_val < 0.01:
            return '**'
        elif p_val < 0.05:
            return '*'
        else:
            return 'ns'
    
    if apply_fdr:
        results_df['Significance'] = results_df['p_value_FDR'].apply(
            lambda x: get_significance_symbol(x, fdr=True)
        )
    else:
        results_df['Significance'] = results_df['p_value'].apply(
            lambda x: get_significance_symbol(x, fdr=False)
        )
    
    return results_df


def print_summary(results_df, apply_fdr=True):
    """
    affiche un résumé des résultats statistiques
    
    parameters:
    results_df : pandas.DataFrame
        DataFrame avec les résultats
    apply_fdr : bool
        Si True, utilise les p-values corrigées FDR
    """
    print("\n" + "="*80)
    print("RÉSUMÉ DES RÉSULTATS")
    print("="*80)
    
    if apply_fdr:
        sig_col = 'Significant_FDR'
        p_col = 'p_value_FDR'
        print("Correction FDR (Benjamini-Hochberg) appliquée")
    else:
        sig_col = 'Significant'
        p_col = 'p_value'
        print("Pas de correction pour comparaisons multiples")
    
    print(f"\nNombre total de ROIs testées: {len(results_df)}")
    
    if apply_fdr:
        n_sig = results_df[sig_col].sum()
    else:
        n_sig = (results_df[p_col] < 0.05).sum()
    
    print(f"ROIs significatives (p < 0.05): {n_sig}")
    print(f"ROIs non significatives: {len(results_df) - n_sig}")
    
    # niveaux de significativité
    n_p001 = (results_df[p_col] < 0.001).sum()
    n_p01 = ((results_df[p_col] >= 0.001) & (results_df[p_col] < 0.01)).sum()
    n_p05 = ((results_df[p_col] >= 0.01) & (results_df[p_col] < 0.05)).sum()
    
    print(f"\nNiveaux de significativité:")
    print(f"  *** (p < 0.001): {n_p001}")
    print(f"  **  (p < 0.01):  {n_p01}")
    print(f"  *   (p < 0.05):  {n_p05}")
    
    print("\n" + "-"*80)
    print("ROIs significatives:")
    print("-"*80)
    
    if apply_fdr:
        sig_rois = results_df[results_df[sig_col]]
    else:
        sig_rois = results_df[results_df[p_col] < 0.05]
    
    if len(sig_rois) > 0:
        for idx, row in sig_rois.iterrows():
            direction = "ARD > CTRL" if row['Median_ARD'] > row['Median_CTRL'] else "ARD < CTRL"
            print(f"{row['ROI']:30s} {row['Significance']:5s} (p = {row[p_col]:.4f}) [{direction}] r = {row['effect_size_r']:.3f}")
    else:
        print("Aucune ROI significative")
    
    print("\n" + "-"*80)
    print("Test de normalité (Shapiro-Wilk):")
    print("-"*80)
    non_normal_ctrl = (results_df['Shapiro_p_CTRL'] < 0.05).sum()
    non_normal_ard = (results_df['Shapiro_p_ARD'] < 0.05).sum()
    print(f"ROIs non-normales dans CTRL: {non_normal_ctrl}/{len(results_df)}")
    print(f"ROIs non-normales dans ARD: {non_normal_ard}/{len(results_df)}")
    print("→ Utilisation du test de Mann-Whitney justifiée" if (non_normal_ctrl > 0 or non_normal_ard > 0) else "→ Les données sont normales")

# fichiers
CTRL_FILE = 'CTRL_DTPA_Mbq.xlsx' 
ARD_FILE = 'ARD_DTPA_Mbq.xlsx'   

# options d'analyse
APPLY_FDR = True  # True = avec correction FDR, False = sans correction
ALPHA = 0.05      # seuil de significativité alpha

# sortie
OUTPUT_FILE = 'mann_whitney_results_DTPA.xlsx' 

print("\n" + "="*80)
print(" ANALYSE MANN-WHITNEY - COMPARAISON CTRL vs ARD - DTPA")
print("="*80)

# charger les données
df_ctrl, df_ard = load_data(CTRL_FILE, ARD_FILE)

# identification des colonnes 
roi_cols = get_roi_columns(df_ctrl)
print(f"\n {len(roi_cols)} ROIs identifiées")

# tests statistiques
results_df = mann_whitney_comparison(
    df_ctrl, df_ard, roi_cols, 
    alpha=ALPHA, 
    apply_fdr=APPLY_FDR
)

# résumé
print_summary(results_df, apply_fdr=APPLY_FDR)

# export 
results_df.to_excel(OUTPUT_FILE, index=False)
print(f"\n Résultats sauvegardés dans: {OUTPUT_FILE}")

# tableau des résultats
print("\n" + "="*80)
print("TABLEAU RÉCAPITULATIF")
print("="*80)
display(results_df[['ROI', 'Significance', 'p_value', 'p_value_FDR', 'Median_CTRL', 'Median_ARD', 'effect_size_r']])

# Standardisation par z-scores robustes

In [ ]:
import pandas as pd
import numpy as np

fichier = "CTRL_DTPA.xlsx"

df = pd.read_excel(fichier)

# identification des colonnes de régions (hors Sujet_ID et Joue)
regions = [col for col in df.columns if col not in ['Sujet_ID', 'Joue']]

# fonction z-scores robuste (MAD)
def robust_zscore_mad(data):
    """
    calcule les z-scores robustes basés sur la médiane et le MAD.
    
    Args:
        data: array ou Series de valeurs
    
    Returns:
        z-scores robustes
    """
    # toutes les valeurs en un seul vecteur (applatissement)
    all_values = data.values.flatten()
    all_values = all_values[~np.isnan(all_values)]  # NaN
    
    # calcul médiane du groupe (sur toutes les ROI, tous les sujets)
    median_group = np.median(all_values)
    
    # calcul MAD (Median Absolute Deviation)
    mad = np.median(np.abs(all_values - median_group))
    
    # facteur de normalisation pour rendre MAD comparable à sd
    mad_normalized = mad * 1.4826
    
    # protection contre MAD = 0
    if mad_normalized == 0:
        print(" Attention : MAD = 0, utilisation de IQR comme fallback")
        q75 = np.percentile(all_values, 75)
        q25 = np.percentile(all_values, 25)
        iqr = q75 - q25
        if iqr == 0:
            return data - median_group  # Retourner différences brutes
        return (data - median_group) / iqr
    
    # calcul z-scores robustes
    z_scores = (data - median_group) / mad_normalized
    
    return z_scores

# standardisation
df_zscores = df.copy()

# application de la transformation sur toutes les régions 
df_zscores[regions] = robust_zscore_mad(df[regions])

# verification
all_z = df_zscores[regions].values.flatten()
all_z = all_z[~np.isnan(all_z)]

print("\n" + "="*60)
print("VÉRIFICATION DE LA STANDARDISATION")
print("="*60)
print(f"Médiane des z-scores : {np.median(all_z):.6f}  (attendu ≈ 0)")
print(f"MAD des z-scores     : {np.median(np.abs(all_z - np.median(all_z))) * 1.4826:.6f}  (attendu ≈ 1)")
print(f"Moyenne des z-scores : {np.mean(all_z):.6f}")
print(f"Écart-type z-scores  : {np.std(all_z):.6f}")
print(f"Min z-score          : {np.min(all_z):.3f}")
print(f"Max z-score          : {np.max(all_z):.3f}")
print(f"% |z| > 2            : {100 * np.sum(np.abs(all_z) > 2) / len(all_z):.1f}%  (attendu ≈ 5%)")
print("="*60 + "\n")

# export
nom_sortie = fichier.replace("_ratios.xlsx", "_zscores.xlsx")
df_zscores.to_excel(nom_sortie, index=False)

print(f" Z-scores robustes sauvegardés dans : {nom_sortie}")
print(f" Dimensions : {df_zscores.shape[0]} sujets × {len(regions)} ROI")

# Matrice de Spearman

## Creation de la matrice

In [ ]:
import pandas as pd
from scipy.stats import spearmanr

# chargement des données z-score (par sujet × région)
df = pd.read_excel("CTRL_DTPA.xlsx", index_col=0)

# calcul de la matrice de corrélation 
corr_matrix, pval_matrix = spearmanr(df, axis=0)

# convertion en DataFrame
corr_df = pd.DataFrame(corr_matrix, index=df.columns, columns=df.columns)
pval_df = pd.DataFrame(pval_matrix, index=df.columns, columns=df.columns)

# export
corr_df.to_excel("CTRL_DTPA_corr_spearman.xlsx")
pval_df.to_excel("CTRL_DTPA_pval_spearman.xlsx")

print(" Matrices Spearman exportées pour CTRL_DTPA")

## Calcul des differences matricielles

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr

# importation des matrice 
ctrl = pd.read_excel("CTRL_DTPA_corr_spearman.xlsx", index_col=0)
ard = pd.read_excel("ARD_DTPA_corr_spearman.xlsx", index_col=0)

# matrices differentielles
diff_ctrl_ard = ctrl - ard

# export
diff_ctrl_ard.to_excel("DTPA_DIFF_CTRL-ARD_spearman.xlsx")

print(" Matrices différentielles DTPA (Spearman) exportées")

# correlation entre matrices
def compare_matrices(mat1, mat2, label):
    vals1 = mat1.values.flatten()
    vals2 = mat2.values.flatten()
    rho, p = spearmanr(vals1, vals2)
    print(f"{label} : Similarité Spearman entre matrices : ρ = {rho:.3f}, p = {p:.6f}")

compare_matrices(ctrl, ard, "DTPA CTRL vs ARD")

# Matrice de Pearson

## Creation des matrices

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import t

fichier = "CTRL_DTPA.xlsx" 

df = pd.read_excel(fichier)
regions = df.columns[1:] 
X = df[regions].to_numpy(dtype=float)

R = np.corrcoef(X, rowvar=False)

n = X.shape[0]  
dfree = n - 2
R_clipped = np.clip(R, -0.999999, 0.999999)  
T = R_clipped * np.sqrt(dfree / (1 - R_clipped**2))
P = 2 * t.sf(np.abs(T), dfree) 

corr_file = fichier.replace(".xlsx", "_corr_pearson.xlsx")
pval_file = fichier.replace(".xlsx", "_pval_pearson.xlsx")

pd.DataFrame(R, index=regions, columns=regions).to_excel(corr_file)
pd.DataFrame(P, index=regions, columns=regions).to_excel(pval_file)

print(f" Matrice de corrélation Pearson sauvegardée : {corr_file}")
print(f" Matrice de p-values sauvegardée : {pval_file}")

## Calcul des matrices

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

ctrl = pd.read_excel("CTRL_DTPA_corr_pearson.xlsx", index_col=0)
ard = pd.read_excel("ARD_DTPA_corr_pearson.xlsx", index_col=0)

diff_ctrl_ard = ctrl - ard

diff_ctrl_ard.to_excel("DTPA_DIFF_CTRL-ARD.xlsx")

print(" Matrices différentielles DTPA exportées")

def compare_matrices(mat1, mat2, label):
    vals1 = mat1.values.flatten()
    vals2 = mat2.values.flatten()
    r, p = pearsonr(vals1, vals2)
    print(f"{label} : Similarité Pearson entre matrices : r = {r:.3f}, p = {p:.4f}")

compare_matrices(ctrl, ard, "DTPA CTRL vs ARD")

# Topologie des réseaux de corrélation par groupe

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.cm as cm
from matplotlib.colors import Normalize

fichier_corr = "CTRL_DTPA_corr_spearman.xlsx"
titre = "CTRL - DTPA (Perméabilité BBB)"
seuil_corr = 0.70  # thresholding

# parametres visuels
figsize = (16, 16)
node_radius = 0.035
edge_width_scale = 6
font_size = 9
label_distance = 0.10

# importation des matrices de corrélation
print("="*70)
print("GÉNÉRATION CONNECTOGRAMME CIRCULAIRE")
print("="*70)
print(f"Fichier : {fichier_corr}")
print(f"Seuil de corrélation : |ρ| ≥ {seuil_corr}")

df_corr = pd.read_excel(fichier_corr, index_col=0)

# liste des ROI
roi_names = df_corr.index.tolist()

# ordre sagittal d'affichage (gauche vers la droite)
roi_order_sagittal = [
    "Ctx_Frontal",
    "Olfactory_Bulb",
    "Ctx_Temporal",
    "Amygdala",
    "Hippocampus",
    "Caudate_Putamen",
    "Thalamus_Hypothalamus",
    "Ctx_Cingulate",
    "Ctx_Parietal",
    "Ctx_Occipital",
    "Cerebellum",
    "WM_Cerebellum",
    "Brain_Stem",
    "WM_Ctx",
    "Ventricle_Lateral",
    "Ventricle_III_IV",
]

# on garde seulement les ROI présents
ordered_roi = [r for r in roi_order_sagittal if r in roi_names] + \
              [r for r in roi_names if r not in roi_order_sagittal]

# angle de départ (180° = à gauche, pour le cortex frontal) puis sens horaire
start_angle_deg = 180.0
angles_ordered = np.linspace(
    np.deg2rad(start_angle_deg),
    np.deg2rad(start_angle_deg) - 2*np.pi,
    len(ordered_roi),
    endpoint=False
)

# dictionnaire angle par ROI (clé = nom du ROI) 
angle_by_roi = {roi: ang for roi, ang in zip(ordered_roi, angles_ordered)}

n_roi = len(roi_names)

print(f"\n Matrice chargée : {df_corr.shape}")
print(f"   Nombre de ROI : {n_roi}")
print(f"   Corrélation min : {df_corr.min().min():.3f}")
print(f"   Corrélation max : {df_corr.max().max():.3f}")

# extraction des aretes
edges = []
for i in range(n_roi):
    for j in range(i+1, n_roi):
        corr = df_corr.iloc[i, j]
        if abs(corr) >= seuil_corr:
            edges.append({
                'source': roi_names[i],
                'target': roi_names[j],
                'correlation': corr,
                'abs_corr': abs(corr)
            })

edges_df = pd.DataFrame(edges)
n_connections = len(edges_df)
n_possible = int(n_roi * (n_roi - 1) / 2)

print(f"\n Connexions extraites : {n_connections}/{n_possible} ({100*n_connections/n_possible:.1f}%)")

if n_connections == 0:
    print(f"\n ATTENTION : Aucune connexion au seuil {seuil_corr}")
    print("   → Réduisez le seuil (ex: 0.6 ou 0.5)")
    exit()

print(f"   Corrélations positives : {sum(edges_df['correlation'] > 0)}")
print(f"   Corrélations négatives : {sum(edges_df['correlation'] < 0)}")

# positions circulaires des ROI (vue sagittale)
radius = 1.0
positions = {}
for roi in roi_names:
    ang = angle_by_roi.get(roi, 0.0) 
    positions[roi] = (radius * np.cos(ang), radius * np.sin(ang))


# couleurs des noeuds par regions anatomique
def assign_node_color(roi_name):
    """Attribue couleur selon région anatomique"""
    if 'Ctx_Frontal' in roi_name or 'Frontal' in roi_name:
        return '#e74c3c'  # Rouge - Cortex Frontal
    elif 'Ctx_Parietal' in roi_name or 'Parietal' in roi_name:
        return '#3498db'  # Bleu - Cortex Pariétal
    elif 'Ctx_Temporal' in roi_name or 'Temporal' in roi_name:
        return '#2ecc71'  # Vert - Cortex Temporal
    elif 'Ctx_Occipital' in roi_name or 'Occipital' in roi_name:
        return '#f39c12'  # Orange - Cortex Occipital
    elif 'Ctx_Cingulate' in roi_name or 'Cingulate' in roi_name:
        return '#9b59b6'  # Violet - Cortex Cingulaire
    elif 'Hippocampus' in roi_name or 'Amygdala' in roi_name:
        return '#e67e22'  # Orange foncé - Système limbique
    elif 'Thalamus' in roi_name or 'Hypothalamus' in roi_name:
        return '#1abc9c'  # Turquoise - Diencéphale
    elif 'Cerebellum' in roi_name:
        return '#34495e'  # Gris foncé - Cervelet
    elif 'Caudate' in roi_name or 'Putamen' in roi_name:
        return '#16a085'  # Vert foncé - Noyaux gris centraux
    elif 'Olfactory' in roi_name:
        return '#c0392b'  # Rouge foncé - Bulbes olfactifs
    elif 'Ventricle' in roi_name or 'WM_' in roi_name or 'Brain_Stem' in roi_name:
        return '#95a5a6'  # Gris clair - Substance blanche/ventricules
    else:
        return '#7f8c8d'  # Gris - Autre

node_colors = [assign_node_color(roi) for roi in roi_names]

# creation de la figure
fig, ax = plt.subplots(figsize=figsize, subplot_kw=dict(aspect="equal"))
ax.set_xlim(-1.7, 1.7)
ax.set_ylim(-1.7, 1.7)
ax.axis('off')

# colormap pour gradient des arêtes
norm = Normalize(vmin=seuil_corr, vmax=1.0)
cmap = cm.get_cmap('YlOrRd')  # Jaune (faible) et Rouge (fort)

# tracer les aretes 
for idx, edge in edges_df.iterrows():
    source = edge['source']
    target = edge['target']
    corr = edge['correlation']
    abs_corr = edge['abs_corr']
    
    # positions
    x1, y1 = positions[source]
    x2, y2 = positions[target]
    
    # couleur selon force (gradient)
    if corr > 0:
        color = cmap(norm(abs_corr))  # jaune-rouge pour positives
    else:
        color = '#1f77b4'  # Bleu pour négatives
    
    # epaisseur selon force de corrélation + mise à l'échelle progressive
    width_factor = (abs_corr - seuil_corr) / (1 - seuil_corr)
    width = 0.8 + width_factor * (edge_width_scale - 0.8)
    
    # transparence selon force
    alpha = 0.4 + width_factor * 0.5
    
    # tracer l'arête
    ax.plot([x1, x2], [y1, y2], 
            color=color, 
            linewidth=width, 
            alpha=alpha, 
            zorder=1,
            solid_capstyle='round')

# tracet les noeuds
for i, roi in enumerate(roi_names):
    x, y = positions[roi]
    
    # cercle pour le nœud
    circle = plt.Circle((x, y), node_radius, 
                       color=node_colors[i], 
                       ec='black', 
                       linewidth=2, 
                       zorder=3)
    ax.add_patch(circle)
    
    # position et rotation du texte
    angle_deg = np.degrees(angle_by_roi[roi])
    
    if -90 < angle_deg < 90:
        # à droite du cercle
        ha = 'left'
        x_text = x + label_distance
        rotation = angle_deg
    else:
        # à gauche du cercle
        ha = 'right'
        x_text = x - label_distance
        rotation = angle_deg - 180
    
    # texte du label
    ax.text(x_text, y, roi_label, 
            fontsize=font_size, 
            ha=ha, 
            va='center',
            rotation=rotation,
            rotation_mode='anchor',
            zorder=4,
            fontweight='bold')

# colorbar pour le gradient
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.025, pad=0.04, aspect=30)
cbar.set_label('Force de corrélation (|ρ|)', rotation=270, labelpad=25, fontsize=13, fontweight='bold')
cbar.ax.tick_params(labelsize=11)

# légende des groupes anatomiques
legend_patches = [
    mpatches.Patch(color='#e74c3c', label='Cortex Frontal'),
    mpatches.Patch(color='#3498db', label='Cortex Pariétal'),
    mpatches.Patch(color='#2ecc71', label='Cortex Temporal'),
    mpatches.Patch(color='#f39c12', label='Cortex Occipital'),
    mpatches.Patch(color='#9b59b6', label='Cortex Cingulaire'),
    mpatches.Patch(color='#e67e22', label='Hippocampe/Amygdale'),
    mpatches.Patch(color='#1abc9c', label='Thalamus/Hypothalamus'),
    mpatches.Patch(color='#34495e', label='Cervelet'),
    mpatches.Patch(color='#16a085', label='Noyaux gris centraux'),
    mpatches.Patch(color='#95a5a6', label='Substance blanche/Ventricules'),
]

legend1 = ax.legend(handles=legend_patches, 
                   loc='upper left', 
                   bbox_to_anchor=(-0.02, 1.02),
                   fontsize=10,
                   framealpha=0.95,
                   title='Régions anatomiques',
                   title_fontsize=11,
                   edgecolor='black',
                   fancybox=True)

# épaisseur des arêtes
legend_lines = [
    Line2D([0], [0], color='gray', linewidth=1, label=f'|ρ| ≈ {seuil_corr:.2f}'),
    Line2D([0], [0], color='gray', linewidth=3, label=f'|ρ| ≈ {(1+seuil_corr)/2:.2f}'),
    Line2D([0], [0], color='gray', linewidth=5, label=f'|ρ| ≈ 1.00'),
]

legend2 = ax.legend(handles=legend_lines,
                   loc='lower left',
                   bbox_to_anchor=(-0.02, -0.02),
                   fontsize=10,
                   framealpha=0.95,
                   title='Épaisseur des arêtes',
                   title_fontsize=11,
                   edgecolor='black',
                   fancybox=True)

ax.add_artist(legend1)

plt.title(titre, fontsize=22, fontweight='bold', pad=30)

# calcul des statistiques réseau
degree_count = {}
for roi in roi_names:
    degree_count[roi] = sum((edges_df['source'] == roi) | (edges_df['target'] == roi))

degree_mean = np.mean(list(degree_count.values()))
degree_std = np.std(list(degree_count.values()))
mean_corr = edges_df['abs_corr'].mean()

# encadrer avec statistiques
stats_text = f"Connexions affichées : {n_connections}/{n_possible} ({100*n_connections/n_possible:.1f}%)\n"
stats_text += f"Degré moyen : {degree_mean:.1f} ± {degree_std:.1f} connexions/nœud\n"
stats_text += f"Force moyenne : |ρ| = {mean_corr:.3f}"

ax.text(0, -1.52, stats_text, 
        fontsize=12, 
        ha='center',
        bbox=dict(boxstyle='round,pad=0.8', facecolor='wheat', alpha=0.5, edgecolor='black', linewidth=1.5))

plt.tight_layout()

# export
output_file = fichier_corr.replace('.xlsx', f'_connectogram_thresh{seuil_corr:.2f}.png')
plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')

print(f"\n Connectogramme sauvegardé : {output_file}")

plt.show()

# statistiques réseau detaillées
print("\n" + "="*70)
print("STATISTIQUES DU RÉSEAU")
print("="*70)

print(f"\nSeuil de corrélation : |ρ| ≥ {seuil_corr}")
print(f"Nombre total de connexions : {n_connections}/{n_possible} ({100*n_connections/n_possible:.1f}%)")
print(f"Densité du réseau : {2*n_connections/(n_roi*(n_roi-1)):.3f}")

print(f"\nForce des connexions :")
print(f"  Moyenne : |ρ| = {mean_corr:.3f}")
print(f"  Médiane : |ρ| = {edges_df['abs_corr'].median():.3f}")
print(f"  Min : |ρ| = {edges_df['abs_corr'].min():.3f}")
print(f"  Max : |ρ| = {edges_df['abs_corr'].max():.3f}")

print(f"\nDegré des nœuds (connectivité) :")
print(f"  Moyenne : {degree_mean:.1f} ± {degree_std:.1f}")
print(f"  Médiane : {np.median(list(degree_count.values())):.0f}")
print(f"  Min : {min(degree_count.values())}")
print(f"  Max : {max(degree_count.values())}")

# Top 5 hubs (régions les plus connectées)
top_hubs = sorted(degree_count.items(), key=lambda x: x[1], reverse=True)[:5]
print("\n TOP 5 HUBS (régions les plus connectées) :")
for rank, (roi, degree) in enumerate(top_hubs, 1):
    print(f"  {rank}. {roi:35s} : {degree:3d} connexions ({100*degree/(n_roi-1):.1f}%)")

# Bottom 5 (régions les moins connectées)
bottom_5 = sorted(degree_count.items(), key=lambda x: x[1])[:5]
print("\n BOTTOM 5 (régions les moins connectées) :")
for rank, (roi, degree) in enumerate(bottom_5, 1):
    print(f"  {rank}. {roi:35s} : {degree:3d} connexions ({100*degree/(n_roi-1):.1f}%)")

# distribution des connexions positives/négatives
n_positive = sum(edges_df['correlation'] > 0)
n_negative = sum(edges_df['correlation'] < 0)
print(f"\nDistribution des corrélations :")
print(f"  Positives : {n_positive} ({100*n_positive/n_connections:.1f}%)")
print(f"  Négatives : {n_negative} ({100*n_negative/n_connections:.1f}%)")

# Fisher-z : Identification des connexions différentielles

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from statsmodels.stats.multitest import multipletests
import matplotlib.cm as cm
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.colors import TwoSlopeNorm
plt.rcParams.update({'font.family': 'Helvetica', 'pdf.fonttype': 42})

LABEL_MAP = {
    "Olfactory_Bulb": "Olfactory Bulb",
    "WM_Cerebellum": "Cerebellar WM",
    "Ventricle_III_IV": "III–IV Ventricles",
    "Ventricle_Lateral": "Lateral Ventricles",
    "Brain_Stem": "Brainstem",
    "WM_Ctx": "Subcortical WM",
    "Ctx_Parietal": "Parietal Cortex",
    "Ctx_Occipital": "Occipital Cortex",
    "tx_Temporal": "Temporal Cortex",
    "Ctx_Frontal": "Frontal Cortex",
    "Ctx_Cingulate": "Cingulate Cortex",
    "Cerebellum": "Cerebellum",
    "Amygdala": "Amygdala",
    "Hippocampus": "Hippocampus",
    "Thalamus_Hyppothalamus": "Thalamus – Hypothalamus",
    "Caudate_Putamen": "Caudate – Putamen",
}

roi_order_sagittal = [
    "Thalamus_Hyppothalamus","Olfactory_Bulb","Ctx_Frontal","WM_Ctx","tx_Temporal","Caudate_Putamen","Amygdala",
    "Ctx_Parietal","Hippocampus","Ctx_Occipital","Cerebellum","WM_Cerebellum",
    "Brain_Stem","Ventricle_III_IV","Ctx_Cingulate","Ventricle_Lateral",
]

def assign_node_color(roi_name):
    """Couleurs des nœuds (identiques aux autres script)."""
    if ('Frontal' in roi_name or 'Parietal' in roi_name or 'Temporal' in roi_name or
        'Occipital' in roi_name or 'Cingulate' in roi_name):
        return '#3498db' 
    elif ('Hippocampus' in roi_name or 'Amygdala' in roi_name or
          'Thalamus' in roi_name or 'Hypothalamus' in roi_name or
          'Caudate' in roi_name or 'Putamen' in roi_name or
          'Olfactory' in roi_name):
        return '#27ae60' 
    elif 'Cerebellum' in roi_name:
        return '#c0392b' 
    elif 'Brain_Stem' in roi_name or 'Brainstem' in roi_name:
        return '#8b4513' 
    elif 'WM_Ctx' in roi_name or 'Ventricle' in roi_name:
        return '#95a5a6'  
    else:
        return '#7f8c8d'  

# parametres de comparaison 
fichier_groupe1 = "CTRL_DTPA_corr_spearman.xlsx"
fichier_groupe2 = "ARD_DTPA_corr_spearman.xlsx"

n1 = 44  # CTRL
n2 = 36  # ARD

alpha = 0.05          # seuil pour FDR
seuil_diff_min = 0.15  # |Δρ| minimal à afficher

titre = "CTRL vs ARD - DTPA"
groupe1_nom = "CTRL"
groupe2_nom = "ARD"

print("="*70)
print("COMPARAISON STATISTIQUE INTER-GROUPES")
print("="*70)
print(f"Groupe 1 : {groupe1_nom} (n={n1})")
print(f"Groupe 2 : {groupe2_nom} (n={n2})")

# chargement
df_corr1 = pd.read_excel(fichier_groupe1, index_col=0)
df_corr2 = pd.read_excel(fichier_groupe2, index_col=0)

# sanitation
assert df_corr1.shape == df_corr2.shape, "Les matrices doivent avoir la même taille."
roi_names = df_corr1.index.tolist()
n_roi = len(roi_names)
print(f"\n Matrices chargées : {n_roi} ROI")

# statistiques Fisher z
def fisher_z(r):
    r = np.clip(r, -0.9999, 0.9999) 
    return 0.5 * np.log((1 + r) / (1 - r))

z_corr1 = fisher_z(df_corr1.values)
z_corr2 = fisher_z(df_corr2.values)

results = []
se_diff = np.sqrt(1/(n1-3) + 1/(n2-3))  # erreur standard diff de z

for i in range(n_roi):
    for j in range(i+1, n_roi):
        r1 = df_corr1.iloc[i, j]
        r2 = df_corr2.iloc[i, j]
        z1 = z_corr1[i, j]
        z2 = z_corr2[i, j]
        delta_z = z2 - z1
        delta_r = r2 - r1
        z_stat = delta_z / se_diff
        p_value = 2 * (1 - norm.cdf(abs(z_stat)))  # bilatéral

        results.append({
            'ROI_1': roi_names[i],
            'ROI_2': roi_names[j],
            'r_groupe1': r1,
            'r_groupe2': r2,
            'delta_r': delta_r,
            'z_stat': z_stat,
            'p_value': p_value
        })

results_df = pd.DataFrame(results)
n_tests = len(results_df)
print(f"   Nomber of tests : {n_tests}")

# correction FDR BH
print(f"\n Correction FDR (α = {alpha})...")
rejected, p_corrected, _, _ = multipletests(results_df['p_value'], alpha=alpha, method='fdr_bh')
results_df['p_corrected'] = p_corrected
results_df['significant'] = rejected

n_signif = int(rejected.sum())
print(f"   Tests significatifs : {n_signif}/{n_tests} ({100*n_signif/n_tests:.1f}%)")

# filtre de l'affichage : significatifs + amplitude minimale
results_signif = results_df[(results_df['significant']) & (np.abs(results_df['delta_r']) >= seuil_diff_min)].copy()
n_signif_display = len(results_signif)
print(f"   To display (p_FDR<{alpha} and |Δρ|≥{seuil_diff_min}) : {n_signif_display}")

if n_signif_display > 0:
    print(f"\n Δρ mean : {results_signif['delta_r'].mean():.3f} | "
          f"|Δρ|min : {results_signif['delta_r'].abs().min():.3f} | "
          f"|Δρ|max : {results_signif['delta_r'].abs().max():.3f}")
    n_increase = int((results_signif['delta_r'] > 0).sum())
    n_decrease = int((results_signif['delta_r'] < 0).sum())
else:
    n_increase = 0
    n_decrease = 0

# export stats
output_stats = f"comparison_{groupe1_nom}_vs_{groupe2_nom}_DTPA_stats.xlsx"
results_df.to_excel(output_stats, index=False)
print(f"\n Statistiques sauvegardées : {output_stats}")

# visualisation 
if n_signif_display > 0:

    #  paramètres visuels identiques au connectome intra-groupe
    figsize = (16, 16)
    node_radius = 0.035
    edge_width_scale = 6
    font_size = 14
    label_distance = 0.10
    radius = 1.0

    # ordre sagittal + angles (départ à 180°, sens horaire)
    ordered_roi = [r for r in roi_order_sagittal if r in roi_names] + \
                  [r for r in roi_names if r not in roi_order_sagittal]

    start_angle_deg = 180.0
    angles_ordered = np.linspace(
        np.deg2rad(start_angle_deg),
        np.deg2rad(start_angle_deg) - 2*np.pi,
        len(ordered_roi),
        endpoint=False
    )
    angle_by_roi = {roi: ang for roi, ang in zip(ordered_roi, angles_ordered)}

    #  positions
    positions = {roi: (radius*np.cos(angle_by_roi[roi]),
                       radius*np.sin(angle_by_roi[roi]))
                 for roi in roi_names}

    #  couleurs des nœuds
    node_colors = [assign_node_color(roi) for roi in roi_names]

    #  figure
    fig, ax = plt.subplots(figsize=figsize, subplot_kw=dict(aspect="equal"))
    ax.set_xlim(-1.7, 1.7)
    ax.set_ylim(-1.7, 1.7)
    ax.axis('off')

    #  colormap 
    max_abs_diff = results_signif['delta_r'].abs().max()
    if max_abs_diff == 0:
        max_abs_diff = 1e-6
    norm = TwoSlopeNorm(vmin=-max_abs_diff, vcenter=0, vmax=max_abs_diff)
    cmap = plt.colormaps['coolwarm_r']

    # tracer arêtes significatifs
    for _, row in results_signif.iterrows():
        s, t = row['ROI_1'], row['ROI_2']
        delta_r = row['delta_r']
        abs_delta = abs(delta_r)

        x1, y1 = positions[s]
        x2, y2 = positions[t]

        color = cmap(norm(delta_r))
        width_factor = abs_delta / max_abs_diff
        width = 0.8 + width_factor * (edge_width_scale - 0.8)
        alpha = 0.4 + width_factor * 0.5

        ax.plot([x1, x2], [y1, y2],
                color=color, linewidth=width, alpha=alpha,
                zorder=1, solid_capstyle='round')

    # nœuds + labels radiaux horizontaux
    for i, roi in enumerate(roi_names):
        x, y = positions[roi]
        circle = plt.Circle((x, y), node_radius,
                            color=node_colors[i], ec='black', linewidth=2, zorder=3)
        ax.add_patch(circle)

        disp_label = LABEL_MAP.get(roi, roi)
        angle_rad = angle_by_roi[roi]
        angle_deg = np.degrees(angle_rad)
        text_radius = radius + label_distance
        x_text = text_radius * np.cos(angle_rad)
        y_text = text_radius * np.sin(angle_rad)
        ha = 'left' if (-90 < angle_deg <= 90) else 'right'

        ax.text(x_text, y_text, disp_label,
                fontsize=font_size, ha=ha, va='center',
                rotation=0, zorder=4, fontweight='bold')

    # colorbar
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, fraction=0.025, pad=0.04, aspect=30)
    cbar.set_label(f'Δρ Difference ({groupe2_nom} - {groupe1_nom})',
                   rotation=270, labelpad=25, fontsize=13, fontweight='bold')
    cbar.ax.tick_params(labelsize=11)

    # légende unique
    legend_lines = [
        Line2D([0], [0], color='#2166ac', linewidth=3,
               label=f'{groupe2_nom} > {groupe1_nom} (Increase)'),
        Line2D([0], [0], color='#b2182b', linewidth=3,
               label=f'{groupe2_nom} < {groupe1_nom} (Decrease)'),
    ]

    ax.legend(handles=legend_lines,
              loc='upper left', bbox_to_anchor=(-0.02, 1.02),
              fontsize=11, framealpha=0.95,
              edgecolor='black', fancybox=True)

    # titre et stats
    plt.title(titre + "\n(Significant differences (FDR < 0.05)",
              fontsize=22, fontweight='bold')

    stats_text = (
        f"Different connections (displayed): {n_signif_display}/{n_tests} "
        f"({100*n_signif_display/n_tests:.1f}%)\n"
        f"Increase : {n_increase} | Decrease : {n_decrease}\n"
        f"Mean |Δρ| : {results_signif['delta_r'].abs().mean():.3f}"
    )
    ax.text(0, -1.52, stats_text, fontsize=12, ha='center',
            bbox=dict(boxstyle='round,pad=0.8', facecolor='wheat',
                      alpha=0.5, edgecolor='black', linewidth=1.5))

    plt.tight_layout()

    # export
    output_fig = f"comparison_{groupe1_nom}_vs_{groupe2_nom}_DTPA_connectogram.png"
    plt.savefig(output_fig, dpi=600, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    print(f"\n Connectogramme sauvegardé : {output_fig}")

    plt.show()

else:
    print("\n Pas de connectogramme généré (aucune différence significative)")